<a href="https://colab.research.google.com/github/jinsusong/21-study-tensorflow/blob/main/%EB%8D%94%EB%AF%B8%20%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A5%BC%20%ED%99%9C%EC%9A%A9%ED%95%9C%20%EA%B0%90%EC%A0%95%20%EB%B6%84%EC%84%9D%20%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텐서플로2와 머신러닝으로 시작하는 자연어 처리 
## 로지스틱 회귀부터 BERT와 GPT2 까지


더미 데이터를 활용한 감정 분석 모델링

- 모델 학습에는 케라스 모델의 내장 API 활용

- 모델은 심층 신경망 DNN 구조를 사용해 텍스트의 긍정/부정을 예측하는 감정 분석 모델

1. 각 단어로 구성된 입력값은 임베딩된 벡터로 변형된다. 
2. 각 벡터를 평균해서 하나의 벡터로 만든다. 
3. 하나의 은닉층을 거친 후 하나의 결과값을 뽑는 구조 
4. 결과값에 시그모이드 함수를 적용해 0과 1 사이의 값을 구함 


In [ ]:
import tensorflow as tf
from tensorflow.keras import preprocessing

samples = ['너 오늘 이뻐 보인다',
           '나는 오늘 기분이 더러워',
           '끝내주는데, 좋은 일이 있나봐',
           '나 좋은 일이 생겼어',
           '아 오늘 진짜 짜증나',
           '환상적인데, 정말 좋은거 같아'
           ]

labels = [[1],[0],[1],[1],[0],[1]]

tokenizer = preprocessing.text.Tokenizer()

tokenizer.fit_on_texts(samples)

sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index



- 모델 구축 및 모델 학습에 필요한 변수 정의

In [ ]:
batch_size = 2
num_epochs = 100
vocab_size = len(word_index) +1
emb_size = 128
hidden_dimension = 256
output_dimension = 1



- 학습 과정에서 적용할 배치 사이즈와 에폭 수, 모델의 하이퍼파라미터에 해당하는 여러 차원의 크기(임베딩 층,은닉 층, 출력 층)

- Sequential API를 활용해 심층 신경망 모델을 생성하려면 

In [ ]:
model = tf.keras.Sequential([
    layers.Embedding(vocab_size, emb_size, input_length =4),
    layers.Lambda(lambda x : tf.reduce_mean(x,axis =1)),
    layers.Dense(hidden_dimension, activation='relu'),
    layers.Dense(output_dimension, activation='sigmoid')])


1. 입력값을 임베딩하는 Embedding 층을 모델에 추가 

2. 임베딩된 단어의 벡터를 평균하기 위해 람다 층을 사용 : 람다 층은 텐서플로 연산을 Sequential API와 Functional API에 적용하기 위해 사용하는 방법

3. 람다 층을 활용해 평균을 낸 후 하나의 은닉층을 통과한 후 최종 출력값을 뽑기 위해 두개의 Dense 층을 모델에 추가

4. 최종 출력값을 뽑은 Dense 층의 경우 0과 1 사이의 확률값을 뽑기 위해 활성화 함수를 시그모이드 함수로 정의




 - 구성된 모델 학습하기 


- 학습 과정 정의 하기 


- 옵티마이저의 경우 : 아담 최적화 알고리즘 사용

- 학습 : 이진 분류 문제이므로 이진 교차 엔트로피 손실 함수 사용 

- 모델 성능 측정을 위한 기준 평가 지표 : 정확도를 평가 지표 정의 





In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

학습 진행 

In [ ]:
model.fit(sequences, labels, epochs=num_epochs, batch_size=batch_size)

In [ ]:
model.summary()

# Functional API 방법으로 동일한 모델 구현 

In [ ]:
import tensorflow as tf
from tensorflow import keras

inputs = layers.Input(shape=(4,))
embed_output = layers.Embedding(vocab_size, emb_size)(inputs)
pooled_output = tf.reduce_mean(embed_output, axis=1)
hidden_layer = layers.Dense(hidden_dimension, activation='relu')(pooled_output)
outputs = layers.Dense(output_dimension, activation='sigmoid')(hidden_layer)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentorpy',
              metrics=['accuracy']
              )
model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)


# Subclassing 방법으로 동일한 모델 구현 


In [ ]:
class CustomModel(tf.keras.Model):

    def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
        super(CustomModel, self).__init__(name='my_model')
        self.embedding = layers.Embedding(vocab_size, embed_dimension)
        self.dense_layer = layers.Dense(hidden_dimension, activation='relu')
        self.output_layer = layers.Dense(output_dimension, activation='sigmoid')

    
    def call(self, inputs):
        x = self.embedding(inputs)
        x = tf.reduce_mean(x, axis=1)
        x = self.dense_layer(x)
        x = self.output_layer(x)

        return x

model = CustomModel(vocab_size = vocab_size,
                    embed_dimension=emb_size,
                    hidden_dimension=hidden_dimension,
                    output_dimension=output_dimension
                    )

model.compile(optimizer= tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentorpy',
              metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)
